In [2]:
from tqdm import tqdm

Get data

In [3]:
datapath = 'data/wiki-pages-text/'

**`sPacy`** - Helper Functions
https://spacy.io/usage/spacy-101#annotations-ner

In [4]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [13]:
def POStag(string):
    return [(word.text, word.pos_) for word in nlp(string)]

def NEtag(string, keywordMode=True):
    if keywordMode:
        return [ent.text for ent in nlp(string).ents]
    else:
        return [(ent.text, ent.label_) for ent in nlp(string).ents]

Create information DF for all shards

Aggregating based on Page_Tiles

Precomputation: Find Keywords for each line, will take long

### _`dump`_
contains the aggregated dataframe in the form of a list

# XAPIAN

In [5]:
import xapian
import json

In [6]:
dbpath = 'index/xIndex'

Xapian build Index

In [7]:
!xapian-delve $dbpath

UUID = e43d8d88-5f65-4750-8169-e5cfa8350b82
number of documents = 5396106
average document length = 210.05
document length lower bound = 1
document length upper bound = 71671
highest document id ever used = 5396106
has positional information = true
revision = 540
currently open for writing = false


## Search

In [8]:
import pandas as pd

In [35]:
TOP_RESULTS_LIMIT = 100

def search(dbpath, querystring, offset=0, pagesize=10):
    
    database = xapian.Database(dbpath)
    enquire = xapian.Enquire(database)
    query_string = querystring

    qp = xapian.QueryParser()
    stemmer = xapian.Stem("english")
    qp.set_stemmer(stemmer)
    qp.set_database(database)
    
    qp.set_stemming_strategy(xapian.QueryParser.STEM_SOME)
    query = qp.parse_query(query_string)

    enquire.set_query(query)
    matches = enquire.get_mset(0, TOP_RESULTS_LIMIT)
    
    resultArray = []
    for m in matches:
        resultArray.append([m.rank + 1, m.percent, m.docid, m.document.get_data().decode('utf-8')])
    return pd.DataFrame.from_records(resultArray, columns=['Rank','Percent','Document_ID','Document_data'], index='Rank')

In [44]:
query = "Murda Beatz's real name is Marshall Mathers."
search(dbpath, query)

,Percent,Document_ID,Document_data
Rank,,,
1,71,3162893,Marshall_Mathers_-LRB-disambiguation-RRB-
2,51,3189018,Mathers
3,46,4448136,Slim_Shady_-LRB-disambiguation-RRB-
4,45,2678157,Kim_-LRB-song-RRB-
5,42,3062115,MMLP
6,42,4786941,The_Marshall_Mathers_LP_2
7,40,4823990,The_Way_I_Am_-LRB-Eminem_song-RRB-
8,39,3419212,"My_Son_Marshall,_My_Son_Eminem"
9,37,4458340,So_Far...
